In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
books = pd.read_csv("BX-Books.csv", delimiter=",", encoding="latin-1")
users = pd.read_csv("BX-Users.csv", delimiter=",", encoding="latin-1")
ratings = pd.read_csv("BX-Book-Ratings.csv", delimiter=",", encoding="latin-1")

/var/folders/zd/sx8dgzl54sg4qlvk7_6wkkzm0000gn/T/ipykernel_10051/3201972648.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("BX-Books.csv", delimiter=",", encoding="latin-1")
/var/folders/zd/sx8dgzl54sg4qlvk7_6wkkzm0000gn/T/ipykernel_10051/3201972648.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  users = pd.read_csv("BX-Users.csv", delimiter=",", encoding="latin-1")


In [3]:
books = books.dropna()
users = users.dropna()
ratings = ratings.dropna()

In [4]:
unique_users = ratings["user_id"].nunique()
unique_books = ratings["isbn"].nunique()

print(f"Number of unique users: {unique_users}")
print(f"Number of unique books: {unique_books}")

Number of unique users: 95513
Number of unique books: 322102


In [5]:
isbn_mapping = {book: i for i, book in enumerate(ratings["isbn"].unique())}
ratings["isbn"] = ratings["isbn"].map(isbn_mapping)

In [6]:
user_id_mapping = {user: i for i, user in enumerate(ratings["user_id"].unique())}
ratings["user_id"] = ratings["user_id"].map(user_id_mapping)

In [7]:
ratings_pivot = ratings.pivot(index="user_id", columns="isbn", values="rating").fillna(
    0
)

/var/folders/zd/sx8dgzl54sg4qlvk7_6wkkzm0000gn/T/ipykernel_10051/3852807831.py:1: PerformanceWarning: The following operation may generate 30764928326 cells in the resulting pandas object.
  ratings_pivot = ratings.pivot(index="user_id", columns="isbn", values="rating").fillna(


ValueError: Index contains duplicate entries, cannot reshape

In [ ]:
from surprise import Dataset
from surprise import Reader

reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(ratings[["user_id", "isbn", "rating"]], reader)

trainset, testset = train_test_split(data, test_size=0.2)

In [ ]:
from surprise import KNNBasic, accuracy
from surprise.model_selection import train_test_split

# User-based collaborative filtering
sim_options = {"name": "cosine", "user_based": True}
algo = KNNBasic(sim_options=sim_options)

algo.fit(trainset)
predictions = algo.test(testset)

print("User-based Model : Test Set")
accuracy.rmse(predictions, verbose=True)

# Item-based collaborative filtering
sim_options = {"name": "cosine", "user_based": False}
algo = KNNBasic(sim_options=sim_options)

algo.fit(trainset)
predictions = algo.test(testset)

print("Item-based Model : Test Set")
accuracy.rmse(predictions, verbose=True)